In [1]:
from typing import Dict, Tuple
import nltk

from tqdm import tqdm

import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
from datasets import Dataset
import evaluate
import torch
import torch.nn as nn
from torch.optim import AdamW
from transformers import Trainer, T5Tokenizer, T5ForConditionalGeneration, TrainingArguments
from transformers import StoppingCriteria, StoppingCriteriaList

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {DEVICE}")

Using device cuda


In [3]:
model_path = f"t5-wikismall-mimic-dir/checkpoint-11500"
t5_model = T5ForConditionalGeneration.from_pretrained(model_path)
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\t5\tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you ca

In [4]:
MAX_LENGTH = 512

def summarize_text_t5(text):
    inputs = t5_tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=MAX_LENGTH,
        truncation=True
    )
    inputs = inputs.to(DEVICE)
    len1 = len(inputs[0])
 
    summary_ids = t5_model.generate(
        inputs,
        exponential_decay_length_penalty=((int) (len1 * 0.8), -1.05),
        encoder_repetition_penalty=0.3,
        no_repeat_ngram_size=4,
        max_length=50,
        num_beams=5,
        temperature=0.9,
    )
 
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [5]:
t5_model = t5_model.to(DEVICE)
with open("sample.txt", 'r') as f:
    sample = [l.strip("\n") for l in f.readlines()]
# print(sample)
summary = [summarize_text_t5(l) for l in sample]
for s in summary:
    print(s)

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


A diabetes is a chronic disease that occurs when the pancreas does not produce enough insulin or when the body can not effectively use the insulin it produces. A diabetic pancreatitis is a
Insulin is a hormone that controls blood glucose.
Hyperglycaemia, also called raised blood glucose or raised blood sugar, is a common effect of uncontrolled diabetes and over time leads to serious damage to many of the body's systems, especially the nerves and blood vessels.
